Flavors of BERT

In [3]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 20.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
from transformers import pipeline

In [13]:
nlp = pipeline("fill-mask" ,model = "bert-base-uncased")
print(type(nlp.model))
preds = nlp(f"If you don’t {nlp.tokenizer.mask_token} at the sign, you will get a ticket")
print('If you don’t *** at the sign, you will get a ticket')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>
If you don’t *** at the sign, you will get a ticket
Token:look. Score: 88.05%
Token:stop. Score: 8.10%
Token:stare. Score: 0.59%
Token:glance. Score: 0.24%
Token:work. Score: 0.22%


In [16]:
nlp =  pipeline("fill-mask",model = "roberta-base")

print(type(nlp.model))
preds = nlp(f"if you don't {nlp.tokenizer.mask_token} at the sign,you will get a ticket ")
for p in preds:
    print(f"token : {p['token_str']}. Score: {100*p['score']:,.2f}%")

Device set to use cpu


<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>
token :  look. Score: 50.77%
token :  stop. Score: 31.84%
token :  stay. Score: 3.20%
token :  stand. Score: 2.24%
token :  pay. Score: 1.17%


In [17]:
nlp = pipeline("fill-mask" , model = "distilroberta-base")
print(type(nlp.model))
preds = nlp(f"if you don't {nlp.tokenizer.mask_token} at the sign, you will get a ticket")
for p in preds:
    print(f"Token : {p['token_str']} : Score : {p['score']:,.2f}")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>
Token :  stop : Score : 0.36
Token :  look : Score : 0.10
Token :  stare : Score : 0.08
Token :  stand : Score : 0.03
Token :  arrive : Score : 0.03


In [26]:
pip install --upgrade datasets evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 30.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


BERT for sequence classification

In [27]:
from transformers import Trainer,TrainingArguments,DistilBertForSequenceClassification,DistilBertTokenizerFast,DataCollatorWithPadding,pipeline
from datasets import Dataset 
import evaluate
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [54]:
snips_file = open('data/snip.train.txt', 'rb')
snips_rows = snips_file.readlines()
snips_row[:20]



[b'listen O\n',
 b'to O\n',
 b'westbam B-artist\n',
 b'alumb O\n',
 b'allergic B-album\n',
 b'on O\n',
 b'google B-service\n',
 b'music I-service\n',
 b'PlayMusic\n',
 b'\n',
 b'add O\n',
 b'step B-entity_name\n',
 b'to I-entity_name\n',
 b'me I-entity_name\n',
 b'to O\n',
 b'the O\n',
 b'50 B-playlist\n',
 b'cl\xc3\xa1sicos I-playlist\n',
 b'playlist O\n',
 b'AddToPlaylist\n']

In [57]:
# This code segment parses the snips dataset into a more manageable format
utterances = []
tokenized_utterances = []
labels_for_tokens = []
sequence_labels = []

utterance, tokenized_utterance, label_for_utterances = '', [], []
for snip_row in snips_rows:
    if len(snip_row) == 2:  # skip over rows with no data
        continue
    if ' ' not in snip_row.decode():  # we've hit a sequence label
        sequence_labels.append(snip_row.decode().strip())
        utterances.append(utterance.strip())
        tokenized_utterances.append(tokenized_utterance)
        labels_for_tokens.append(label_for_utterances)
        utterance = ''
        tokenized_utterance = []
        label_for_utterances = []
        continue
    token, token_label = snip_row.decode().split(' ')
    token_label = token_label.strip()
    utterance += f'{token} '
    tokenized_utterance.append(token)
    label_for_utterances.append(token_label)

In [65]:
len(labels_for_tokens), len(tokenized_utterances), len(utterances), len(sequence_labels)
print(labels_for_tokens[0])
print(tokenized_utterances[0],utterances[0],sequence_labels)

['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music'] listen to westbam alumb allergic on google music ['PlayMusic', '', 'AddToPlaylist', '', 'RateBook', '', 'PlayMusic', '', 'AddToPlaylist', '', 'AddToPlaylist', '', 'PlayMusic', '', 'PlayMusic', '', 'SearchScreeningEvent', '', 'BookRestaurant', '', 'GetWeather', '', 'RateBook', '', 'SearchScreeningEvent', '', 'PlayMusic', '', 'GetWeather', '', 'BookRestaurant', '', 'BookRestaurant', '', 'AddToPlaylist', '', 'BookRestaurant', '', 'AddToPlaylist', '', 'SearchScreeningEvent', '', 'AddToPlaylist', '', 'SearchCreativeWork', '', 'SearchCreativeWork', '', 'AddToPlaylist', '', 'SearchScreeningEvent', '', 'SearchCreativeWork', '', 'RateBook', '', 'SearchScreeningEvent', '', 'SearchScreeningEvent', '', 'PlayMusic', '', 'RateBook', '', 'AddToPlaylist', '', 'SearchScreeningEvent', '', 'SearchCreativeWork', '', 'BookRestaurant', '', 'GetWeather', '', 'PlayMus

In [66]:
Tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-cased")

In [67]:
unique_sequence_labels = list(set(sequence_labels))
unique_sequence_labels

['',
 'RateBook',
 'AddToPlaylist',
 'BookRestaurant',
 'PlayMusic',
 'SearchScreeningEvent',
 'SearchCreativeWork',
 'GetWeather']

In [72]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

print(f'There are {len(unique_sequence_labels)} unique sequence labels')

There are 8 unique sequence labels


In [ ]:
from functools import reduce

unique_token_labels = list(set(reduce(lambda x, y: x + y, labels_for_tokens)))
labels_for_tokens = [[unique_token_labels.index(_) for _ in l] for l in labels_for_tokens]

print(f'There are {len(unique_token_labels)} unique token labels')

There are 72 unique token labels
[[69, 69, 17, 69, 22, 69, 70, 59], [], [69, 52, 68, 68, 69, 69, 4, 10, 69], [], [69, 69, 69, 7, 33, 69, 69, 69, 69, 5, 69, 69, 69, 69, 69, 63], [], [69, 69, 60, 3, 58, 58], [], [69, 69, 17, 53, 69, 26, 69, 4, 10, 10], [], [69, 17, 53, 53, 69, 26, 4, 10, 10, 69], [], [69, 69, 69, 69, 69, 23, 69], [], [69, 69, 14, 60, 69, 17, 53], [], [69, 15, 48], [], [69, 69, 69, 69, 13, 69, 32], [], [69, 69, 69, 69, 69, 36, 32, 69, 27, 62, 62, 62, 62, 62, 62], [], [69, 7, 16, 69, 5], [], [69, 69, 69, 69, 69, 33, 40, 69, 18, 28, 64, 64], [], [69, 69, 69, 69, 69, 23, 69, 69, 17], [], [69, 69, 69, 69, 69, 69, 69, 69, 24, 41, 41, 28, 27, 62, 62], [], [69, 69, 66, 69, 13, 69, 27, 62, 62], [], [69, 69, 69, 69, 69, 66, 69, 36, 6, 32, 69, 65, 1, 1, 1], [], [69, 69, 69, 4, 10, 52, 68, 68, 68], [], [69, 69, 66, 69, 27, 62, 62, 69, 32], [], [69, 69, 60, 69, 69, 4, 10, 10, 10, 69], [], [69, 69, 57, 55, 69, 15, 48, 48, 48], [], [69, 52, 68, 69, 26, 69, 4, 10, 10], [], [69, 69, 69, 

In [73]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print([unique_token_labels[l] for l in labels_for_tokens[0]])
print(utterances[0])
print(sequence_labels[0])
print(unique_sequence_labels[sequence_labels[0]])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[69, 69, 17, 69, 22, 69, 70, 59]
[69, 69, 17, 69, 22, 69, 70, 59]
listen to westbam alumb allergic on google music
4
PlayMusic
